<a href="https://colab.research.google.com/github/r0cketr1kky/Median_paper/blob/master/FLavg_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

     |████████████████████████████████| 378kB 2.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 2.0MB 12.0MB/s 
     |████████████████████████████████| 450kB 22.0MB/s 
     |████████████████████████████████| 2.1MB 25.3MB/s 
     |████████████████████████████████| 4.0MB 38.7MB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
     |████████████████████████████████| 1.8MB 42.0MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 1.3MB 41.9MB/s 
     |████████████████████████████████| 36.9MB 88kB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 2.7MB 43.2MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Create

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class Parser:
    """Parameters for training"""
    def __init__(self):
        self.epochs = 10
        self.lr = 0.001
        self.test_batch_size = 8
        self.batch_size = 8
        self.log_interval = 10
        self.seed = 1
    
args = Parser()

torch.manual_seed(args.seed)
kwargs = {}

In [0]:
with open('/gdrive/My Drive/boston_housing.pickle','rb') as f:
    #print(pickle.load(f))
    ((X, y), (X_test, y_test)) = pickle.load(f)
    #print(pickle.load(f))

X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
# preprocessing
mean = X.mean(0, keepdim=True)
dev = X.std(0, keepdim=True)
mean[:, 3] = 0. # the feature at column 3 is binary,
dev[:, 3] = 1.  # so we don't standardize it
X = (X - mean) / dev
X_test = (X_test - mean) / dev
train = TensorDataset(X, y)
test = TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc3 = nn.Linear(24, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
optimizer = optim.SGD(model.parameters(), lr=args.lr)

In [0]:
import syft as sy

hook = sy.TorchHook(torch)
#bob = sy.VirtualWorker(hook, id="bob")
#alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")

x = [0] * 10
for i in range(10):
  x[i] = sy.VirtualWorker(hook, id="{}".format(i))
print(x)
compute_nodes = x

[<VirtualWorker id:0 #objects:0>, <VirtualWorker id:1 #objects:0>, <VirtualWorker id:2 #objects:0>, <VirtualWorker id:3 #objects:0>, <VirtualWorker id:4 #objects:0>, <VirtualWorker id:5 #objects:0>, <VirtualWorker id:6 #objects:0>, <VirtualWorker id:7 #objects:0>, <VirtualWorker id:8 #objects:0>, <VirtualWorker id:9 #objects:0>]


In [0]:
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_distributed_dataset):
        worker = data.location
        model.send(worker)

        optimizer.zero_grad()
        # update the model
        pred = model(data)
        loss = F.mse_loss(pred.view(-1), target)
        loss.backward()
        optimizer.step()
        model.get()
            
        if batch_idx % args.log_interval == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test():
    model.eval()
    test_loss = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

In [0]:
import time
t = time.time()

for epoch in range(1, args.epochs + 1):
    train(epoch)

    
total_time = time.time() - t
print('Total', round(total_time, 2), 's')

Train Epoch: 1 [0/51 (0%)]	Loss: 499.737000
Train Epoch: 1 [80/51 (20%)]	Loss: 444.433716
Train Epoch: 1 [160/51 (39%)]	Loss: 332.781464
Train Epoch: 1 [240/51 (59%)]	Loss: 133.071625
Train Epoch: 1 [320/51 (78%)]	Loss: 202.246475
Train Epoch: 1 [200/51 (98%)]	Loss: 16.212317
Train Epoch: 2 [0/51 (0%)]	Loss: 43.216484
Train Epoch: 2 [80/51 (20%)]	Loss: 8.341543
Train Epoch: 2 [160/51 (39%)]	Loss: 21.476643
Train Epoch: 2 [240/51 (59%)]	Loss: 24.701014
Train Epoch: 2 [320/51 (78%)]	Loss: 135.487076
Train Epoch: 2 [200/51 (98%)]	Loss: 16.858915
Train Epoch: 3 [0/51 (0%)]	Loss: 31.058374
Train Epoch: 3 [80/51 (20%)]	Loss: 6.284317
Train Epoch: 3 [160/51 (39%)]	Loss: 16.468430
Train Epoch: 3 [240/51 (59%)]	Loss: 20.231106
Train Epoch: 3 [320/51 (78%)]	Loss: 109.964317
Train Epoch: 3 [200/51 (98%)]	Loss: 14.596972
Train Epoch: 4 [0/51 (0%)]	Loss: 31.536037
Train Epoch: 4 [80/51 (20%)]	Loss: 6.582751
Train Epoch: 4 [160/51 (39%)]	Loss: 14.647831
Train Epoch: 4 [240/51 (59%)]	Loss: 15.352842


In [0]:
test()


Test set: Average loss: 20.7677



In [0]:
# ENCRYPTED AGGREGATION

In [0]:
import numpy as np

In [97]:
remote_dataset = (list(),list(),list(),list(),list(),list(),list(),list(),list(),list())

train_distributed_dataset = []
print(compute_nodes)
for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))

def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = F.mse_loss(pred.view(-1), target)
    loss.backward()
    optimizer.step()
    return model

#bobs_model = Net()
#alices_model = Net()

models = [0] * 10
optimizers = [0] * 10
params = [0] * 10
for i in range(10):
  models[i] = Net()
  optimizers[i] = optim.SGD(models[i].parameters(), lr=args.lr)
  params[i] = list(models[i].parameters())

#print(len(params))
#print(params[0])
#bobs_optimizer = optim.SGD(bobs_model.parameters(), lr=args.lr)
#alices_optimizer = optim.SGD(alices_model.parameters(), lr=args.lr)

#models = [bobs_model, alices_model]
#params = [list(bobs_model.parameters()), list(alices_model.parameters())]
#optimizers = [bobs_optimizer, alices_optimizer]

[<VirtualWorker id:0 #objects:34>, <VirtualWorker id:1 #objects:8>, <VirtualWorker id:2 #objects:8>, <VirtualWorker id:3 #objects:8>, <VirtualWorker id:4 #objects:8>, <VirtualWorker id:5 #objects:8>, <VirtualWorker id:6 #objects:8>, <VirtualWorker id:7 #objects:8>, <VirtualWorker id:8 #objects:8>, <VirtualWorker id:9 #objects:11>]


In [0]:
# this is selecting which batch to train on
data_index = 0
# update remote models
# we could iterate this multiple times before proceeding, but we're only iterating once per worker here
for remote_index in range(len(compute_nodes)):
    data, target = remote_dataset[remote_index][data_index]
    models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])

In [0]:
# create a list where we'll deposit our encrypted model average
new_params = list()
# iterate through each parameter
for param_i in range(len(params[0])):

    # for each worker
    spdz_params = list()
    for remote_index in range(len(compute_nodes)):
        
        # since SMPC can only work with integers (not floats), we need
        # to use Integers to store decimal information. In other words,
        # we need to use "Fixed Precision" encoding.
        fixed_precision_param = params[remote_index][param_i].fix_precision()
        
        # now we encrypt it on the remote machine. Note that 
        # fixed_precision_param is ALREADY a pointer. Thus, when
        # we call share, it actually encrypts the data that the
        # data is pointing TO. This returns a POINTER to the 
        # MPC secret shared object, which we need to fetch.
        #print(fixed_precision_param)
        #print(len(fixed_precision_param))

        for i in range(9):
          if i % 2 == 0:
            
            
            encrypted_param = fixed_precision_param.share(x[i], x[i+1], crypto_provider=james)

            #print(encrypted_param)
        
            # now we fetch the pointer to the MPC shared value
            param = encrypted_param.get()
        
            # save the parameter so we can average it with the same parameter
            # from the other workers
            spdz_params.append(param)

    # average params from multiple workers, fetch them to the local machine
    # decrypt and decode (from fixed precision) back into a floating point number
    #print(spdz_params)
    
    new_param = 0
    for i in range(10):
      new_param += (spdz_params[i]).get().float_precision()

    new_param /= 10
    #print(new_param)
    # save the new averaged parameter
    new_params.append(new_param)

#print(new_params)
with torch.no_grad():
    for model in params:
        for param in model:
            param *= 0

    for model in models:
        model.get()

    for remote_index in range(len(compute_nodes)):
        for param_index in range(len(params[remote_index])):
            params[remote_index][param_index].set_(new_params[param_index])

In [100]:
def train(epoch):
    for data_index in range(len(remote_dataset[0])-1):
        # update remote models
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])

        # encrypted aggregation
        new_params = list()
        for param_i in range(len(params[0])):
            spdz_params = list()
            for remote_index in range(len(compute_nodes)):
                fixed_precision_param = params[remote_index][param_i].fix_precision()

                for i in range(9):
                  encrypted_param = fixed_precision_param.share(x[i], x[i+1], crypto_provider=james)

                  param = encrypted_param.get()
                
                  spdz_params.append(param)

                
            new_param = 0
            for i in range(10):
              new_param += (spdz_params[i]).get().float_precision()
            
            new_param /= 10
            # save the new averaged parameter
            new_params.append(new_param)


        # cleanup
        with torch.no_grad():
            for model in params:
                for param in model:
                    param *= 0

            for model in models:
                model.get()

            for remote_index in range(len(compute_nodes)):
                for param_index in range(len(params[remote_index])):
                    params[remote_index][param_index].set_(new_params[param_index])
def test():
    models[0].eval()
    test_loss = 0
    for data, target in test_loader:
        output = models[0](data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}\n'.format(test_loss))
t = time.time()

for epoch in range(args.epochs):
    print(f"Epoch {epoch + 1}")
    train(epoch)
    test()

    
total_time = time.time() - t
print('Total', round(total_time, 2), 's')

Epoch 1
Test set: Average loss: 606.7265

Epoch 2
Test set: Average loss: 591.3130

Epoch 3
Test set: Average loss: 572.2477

Epoch 4
Test set: Average loss: 544.2882

Epoch 5
Test set: Average loss: 494.3641

Epoch 6
Test set: Average loss: 387.9724

Epoch 7
Test set: Average loss: 190.3325

Epoch 8
Test set: Average loss: 81.6172

Epoch 9
Test set: Average loss: 55.9037

Epoch 10
Test set: Average loss: 43.2665

Total 77.29 s


In [95]:
print(len(test_loader.dataset))

102
